In [1]:
%pip install inputs

Note: you may need to restart the kernel to use updated packages.


In [5]:
from inputs import devices

print(devices.all_devices)

[inputs.Keyboard("/dev/input/by-id/usb-A_Nice_Keyboard-event-kbd"), inputs.Mouse("/dev/input/by-id/usb-A_Nice_Mouse_called_Arthur-event-mouse")]


In [6]:
%pip install --upgrade setuptools
%pip install hidapi

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import hid

for device in hid.enumerate():
    print(f"0x{device['vendor_id']:04x}:0x{device['product_id']:04x} {device['product_string']}")

0xfeed:0x6060 keyboard
0xfeed:0x6060 keyboard
0x320f:0x5064 USB DEVICE
0x3434:0xd033 Keychron M3 
0x3434:0xd030 Keychron Link 
0x3434:0xd030 Keychron Link 
0x3434:0xd033 Keychron M3 
0x3434:0xd030 Keychron Link 
0x3434:0xd033 Keychron M3 
0xfeed:0x6060 keyboard
0x320f:0x5064 USB DEVICE
0x3434:0xd030 Keychron Link 
0xfeed:0x6060 keyboard
0x048d:0x5702 ITE Device
0x320f:0x5064 USB DEVICE
0x048d:0x5702 ITE Device
0x320f:0x5064 USB DEVICE
0xfeed:0x6060 keyboard
0x320f:0x5064 USB DEVICE
0x3434:0xd030 Keychron Link 
0x320f:0x5064 USB DEVICE
0x3434:0xd030 Keychron Link 
0x0951:0x170f HyperX SoloCast
0x0951:0x170f HyperX SoloCast
0x3434:0xd033 Keychron M3 


In [10]:
gamepad = hid.device()
gamepad.open(0x0079, 0x0011) # The vendor and product IDs of the gamepad
gamepad.set_nonblocking(True)

print(gamepad)

# "Calibration" code to find the bit for each button

In [12]:
BIT_NAMES = {}

In [13]:
from time import sleep

data_wait_count = 0
last_bits = None
while True:
    report = gamepad.read(64)
    if report:
        # Convert the array of 8 0-255 values to a bit array
        bits = [0] * 64
        for i in range(8):
            for j in range(8):
                bits[i * 8 + j] = (report[i] >> j) & 1

        # Only print the bit array if it has changed
        if last_bits != None and bits != last_bits:
            for bit in range(len(bits)):
                if last_bits[bit] != bits[bit]:
                    print(f"Bit {bit} changed to {bits[bit]}")
                    # Ask the user if they want to map this bit to a name
                    if bit not in BIT_NAMES:
                        name = input("Enter a name for this bit: ")
                        BIT_NAMES[bit] = name

        last_bits = bits
        sleep(1 / 60)
    else:
        print("No data...")
        data_wait_count += 1
        sleep(0.1)
        if data_wait_count > 10:
            print("No data for 10 iterations, breaking...")
            break

Bit 46 changed to 1
Bit 46 changed to 0
Bit 47 changed to 1
Bit 47 changed to 0
Bit 44 changed to 1
Bit 44 changed to 0
Bit 45 changed to 1
Bit 45 changed to 0
Bit 53 changed to 1
Bit 53 changed to 0
Bit 52 changed to 1
Bit 52 changed to 0
Bit 39 changed to 1
Bit 39 changed to 0
Bit 51 changed to 1
Bit 51 changed to 0
Bit 50 changed to 1
Bit 50 changed to 0
Bit 49 changed to 1
Bit 49 changed to 0
Bit 48 changed to 1
Bit 48 changed to 0
Bit 48 changed to 1
Bit 48 changed to 0
Bit 45 changed to 1
Bit 45 changed to 0
Bit 49 changed to 1
Bit 49 changed to 0
Bit 44 changed to 1
Bit 44 changed to 0
Bit 46 changed to 1
Bit 39 changed to 1
Bit 46 changed to 0
Bit 39 changed to 0
Bit 47 changed to 1
Bit 47 changed to 0
Bit 51 changed to 1
Bit 51 changed to 0
Bit 46 changed to 1
Bit 46 changed to 0
Bit 46 changed to 1
Bit 46 changed to 0
Bit 50 changed to 1
Bit 50 changed to 0
Bit 52 changed to 1
Bit 52 changed to 0
Bit 52 changed to 1
Bit 52 changed to 0
Bit 53 changed to 1
Bit 53 changed to 0


KeyboardInterrupt: 

In [15]:
print(BIT_NAMES)

# Save the bit names to a file
with open("bit_names.txt", "w") as f:
    for bit, name in BIT_NAMES.items():
        f.write(f"{bit} {name}\n")

{46: 'Up arrow', 47: 'Right arrow', 44: 'Left arrow', 45: 'Down arrow', 53: 'Start', 52: 'Select', 39: 'Center', 51: 'Top right circle', 50: 'Top left X', 49: 'Bottom left triangle', 48: 'Bottom right square'}


In [17]:
gamepad.close() # If not continuing to the execution part, close the device

# Execution code

In [12]:
BIT_NAMES = {}

# Load the bit names from a file
with open("bit_names.txt", "r") as f:
    for line in f:
        bit, name = line.strip().split(" ", 1)
        BIT_NAMES[int(bit)] = name

print(f"Loaded {len(BIT_NAMES)} bit names: {BIT_NAMES}")

Loaded 11 bit names: {46: 'Up arrow', 47: 'Right arrow', 44: 'Left arrow', 45: 'Down arrow', 53: 'Start', 52: 'Select', 39: 'Center', 51: 'Top right circle', 50: 'Top left X', 49: 'Bottom left triangle', 48: 'Bottom right square'}


In [22]:
from IPython.display import display

held_button_display = display("No buttons held", display_id=True)

while True:
    report = gamepad.read(64)
    if report:
        # Print every held button with its name
        held_buttons = []
        for bit, name in BIT_NAMES.items():
            if report[bit // 8] & (1 << (bit % 8)):
                held_buttons.append(name)
        
        if held_buttons:
            held_button_display.update(f"Held buttons: {', '.join(held_buttons)}")
        else:
            held_button_display.update("No buttons held")

        sleep(1.0 / 60)
    else:
        print("No data...")
        sleep(0.1)

'No buttons held'

KeyboardInterrupt: 

In [30]:
%pip install pyntcore

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 12.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/956.9 kB ? eta -:--:--
   --------------------------------------- 956.9/956.9 kB 22.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 18.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
# Actual NetworkTables code

import ntcore
from time import sleep

nt_instance = ntcore.NetworkTableInstance.getDefault()

print(f"Setting up NetworkTables client")
nt_instance.startClient4("NTControls")
nt_instance.setServer("127.0.0.1")
nt_instance.startDSClient()

# Wait for connection
print("Waiting for connection to NetworkTables server...")
while not nt_instance.isConnected():
    sleep(0.1)

print("Connected!")

table = nt_instance.getTable("NTControls")

BUTTON_TO_NT_INPUT = {
    # Manually authored for now
    "Up arrow": "FORWARD",
    "Right arrow": "RIGHT",
    "Left arrow": "LEFT",
    "Down arrow": "BACK",
    "Start": "TURN_LEFT",
    "Select": "TURN_RIGHT",
    "Center": None, # N/A
    "Top right circle": ["FORWARD", "RIGHT"],
    "Top left X": ["FORWARD", "LEFT"],
    "Bottom left triangle": ["BACK", "LEFT"],
    "Bottom right square": ["BACK", "RIGHT"]
}

pressed_buttons = {}

while True:
    report = gamepad.read(64)
    if report:
        # Send every held button with its name to NetworkTables
        for bit, name in BIT_NAMES.items():
            if not name in BUTTON_TO_NT_INPUT or BUTTON_TO_NT_INPUT[name] == None:
                continue

            buttons = BUTTON_TO_NT_INPUT[name]
            if not isinstance(buttons, list):
                buttons = [buttons]
            
            pressed = report[bit // 8] & (1 << (bit % 8))
            for button in buttons:
                if not button in pressed_buttons:
                    pressed_buttons[button] = set()
                
                prev_pressed = len(pressed_buttons[button]) > 0
                if pressed:
                    pressed_buttons[button].add(name)
                else:
                    pressed_buttons[button].discard(name)
                new_pressed = len(pressed_buttons[button]) > 0

                if prev_pressed != new_pressed:
                    print(f"Sending {button} {new_pressed}")
                    table.putBoolean(button, new_pressed)
                
        sleep(1.0 / 60)
    else:
        print("No data...")
        sleep(0.1)

Sending FORWARD True
Sending RIGHT True
Sending FORWARD False
Sending RIGHT False
Sending FORWARD True
Sending FORWARD False
Sending FORWARD True
Sending LEFT True
Sending TURN_RIGHT True
Sending TURN_RIGHT False
Sending FORWARD False
Sending LEFT False
Sending TURN_RIGHT True
Sending TURN_RIGHT False
Sending FORWARD True
Sending LEFT True
Sending LEFT False
Sending FORWARD False
Sending FORWARD True
Sending FORWARD False
Sending RIGHT True
Sending RIGHT False


KeyboardInterrupt: 